# Очистка данных

Ниже приведен анализ входных данных на наличие дубликатов, пропусков и выбросов. Разработанные функции в дальнейшем используются для создания DAGа, который очищает датасет, созданный на прошлом этапе. Код сопровождается комментариями.

Подключение к БД и загрузка датасета:

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table

load_dotenv()

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')
                        
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

sql = f"""
    SELECT *
    FROM real_estate
"""
data = pd.read_sql(sql, dst_conn)

print(data.shape)
print(data.head())

## Дубликаты

Следующий код проверяет, являются ли все 'flat_id' уникальны:

In [ ]:
is_duplicated_id = data.duplicated(subset=['flat_id'], keep=False)
print(sum(is_duplicated_id))

Количество дубликатов равно 0, следовательно, все flat_id уникальны.

Далее, ищем строки, где все переменные равны, но flat_id при этом отличается:

In [ ]:
def remove_duplicates(data):
    feature_cols = data.columns.drop('flat_id').tolist()
    is_duplicated_features = data.duplicated(subset=feature_cols, keep=False)
    data = data[~is_duplicated_features].reset_index(drop=True)
    return data

remove_duplicates(data)

In [ ]:
print(data.shape)

Таковых не обнаружено. Количество строк все еще 141362.

## Пропуски

Количество пропусков в каждой колонке:

In [ ]:
print(data.isnull().sum())

Ни в одной из колонок пропусков не обнаружено.

## Выбросы

Наконец, очистим датасет от наблюдений, которые значительно выбиваются из общего паттерна. Для этого выделим все колонки, которые имеют тип данных 'float':

In [ ]:
num_features = data.select_dtypes(include=['float'])
print(num_features)

Для выполнения задачи воспользуемся методом IRQ:

In [ ]:
threshold = 1.5 # Нормирующий коэффициент
potential_outliers = pd.DataFrame()

for col in num_features:
	Q1 = data[col].quantile(0.25)
	Q3 = data[col].quantile(0.75)
	IQR = Q3 - Q1
	margin = threshold * IQR
	lower = Q1 - margin # Нижняя граница
	upper = Q3 + margin # Верхняя граница
	potential_outliers[col] = ~data[col].between(lower, upper)

outliers = potential_outliers.any(axis=1)

print(data[outliers])

Обнаружено 20674 строки, которые выбиваются из общего ряда. Удалим их из датасета:

In [ ]:
rows_to_remove = data[outliers]['id'].tolist()
print(rows_to_remove)
filtered_df = data[~data['id'].isin(rows_to_remove)]
print(filtered_df.shape)

## Заключение

Итоговый датасет имеет 120688 строк. Данные готовы для использования.